In [1]:
from sklearn.metrics import classification_report
import numpy as np

In [3]:
import pandas as pd
df=messages = pd.read_csv('df_survey_bi.csv')
df.head()

,text,class
0,so jew hating muslim of them those sneaky jew...,hate
1,Stoni is a fuckin queer...,hate
2,they are out smarting u . anither proof had th...,hate
3,if the people of the state don't want faggots ...,hate
4,Soros needs to die like a diseased dog in the ...,hate


In [4]:
#hate=0 not_hate=1

In [5]:
df = df.dropna()

In [6]:
df.shape

(899, 2)

In [7]:
X=list(df['text'])

In [8]:
y=list(df['class'])

In [9]:
y=list(pd.get_dummies(y,drop_first=True)['not_hate'])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [12]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [13]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [14]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [15]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps = 10,
)

In [16]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [17]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.5306433836619059}

In [25]:
#trainer.predict(test_dataset)
#trainer.predict(test_dataset)[0].shape

In [27]:
output = trainer.predict(test_dataset)[0]
output = np.argmax(output, axis = - 1)

In [28]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,output)
cm

array([[43, 40],
       [13, 84]], dtype=int64)

In [29]:
#trainer.save_model('senti_model')

In [30]:
# model's performance
print(classification_report(y_test, output))

              precision    recall  f1-score   support

           0       0.77      0.52      0.62        83
           1       0.68      0.87      0.76        97

    accuracy                           0.71       180
   macro avg       0.72      0.69      0.69       180
weighted avg       0.72      0.71      0.69       180

